# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 18 2023 2:03PM,261348,10,SONSB0002175,"Nextsource Biotechnology, LLC",Released
1,Apr 18 2023 2:00PM,261346,19,9366792,Eywa Pharma Inc.,Released
2,Apr 18 2023 2:00PM,261346,19,9392062,Eywa Pharma Inc.,Released
3,Apr 18 2023 1:53PM,261347,10,MSP220883,"Methapharm, Inc.",Released
4,Apr 18 2023 1:53PM,261347,10,MSP220885,"Methapharm, Inc.",Released
5,Apr 18 2023 1:53PM,261347,10,MSP220886,"Methapharm, Inc.",Released
6,Apr 18 2023 1:53PM,261347,10,MSP220887,"Methapharm, Inc.",Released
7,Apr 18 2023 1:53PM,261347,10,MSP220888,"Methapharm, Inc.",Released
8,Apr 18 2023 1:53PM,261347,10,MSP220889,"Methapharm, Inc.",Released
9,Apr 18 2023 1:53PM,261347,10,MSP220890,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,261344,Released,1
33,261345,Released,1
34,261346,Released,2
35,261347,Released,9
36,261348,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261344,NaN,NaN,1.0
261345,NaN,NaN,1.0
261346,NaN,NaN,2.0
261347,NaN,NaN,9.0
261348,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,16.0,27.0,10.0
261229,0.0,0.0,34.0
261234,0.0,0.0,1.0
261238,5.0,13.0,2.0
261254,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,16,27,10
261229,0,0,34
261234,0,0,1
261238,5,13,2
261254,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,16,27,10
1,261229,0,0,34
2,261234,0,0,1
3,261238,5,13,2
4,261254,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,16,27,10
1,261229,,,34
2,261234,,,1
3,261238,5,13,2
4,261254,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 18 2023 2:03PM,261348,10,"Nextsource Biotechnology, LLC"
1,Apr 18 2023 2:00PM,261346,19,Eywa Pharma Inc.
3,Apr 18 2023 1:53PM,261347,10,"Methapharm, Inc."
12,Apr 18 2023 1:51PM,261345,10,"Methapharm, Inc."
13,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc."
14,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC"
15,Apr 18 2023 1:08PM,261339,15,"Alliance Pharma, Inc."
17,Apr 18 2023 1:08PM,261328,15,"Brookfield Pharmaceuticals, LLC"
18,Apr 18 2023 1:07PM,261326,15,"Person & Covey, Inc."
19,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 18 2023 2:03PM,261348,10,"Nextsource Biotechnology, LLC",,,1
1,Apr 18 2023 2:00PM,261346,19,Eywa Pharma Inc.,,,2
2,Apr 18 2023 1:53PM,261347,10,"Methapharm, Inc.",,,9
3,Apr 18 2023 1:51PM,261345,10,"Methapharm, Inc.",,,1
4,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc.",,,1
5,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC",,,1
6,Apr 18 2023 1:08PM,261339,15,"Alliance Pharma, Inc.",,1,1
7,Apr 18 2023 1:08PM,261328,15,"Brookfield Pharmaceuticals, LLC",,,1
8,Apr 18 2023 1:07PM,261326,15,"Person & Covey, Inc.",,,1
9,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc.",,,34


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 2:03PM,261348,10,"Nextsource Biotechnology, LLC",1,,
1,Apr 18 2023 2:00PM,261346,19,Eywa Pharma Inc.,2,,
2,Apr 18 2023 1:53PM,261347,10,"Methapharm, Inc.",9,,
3,Apr 18 2023 1:51PM,261345,10,"Methapharm, Inc.",1,,
4,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc.",1,,
5,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC",1,,
6,Apr 18 2023 1:08PM,261339,15,"Alliance Pharma, Inc.",1,1,
7,Apr 18 2023 1:08PM,261328,15,"Brookfield Pharmaceuticals, LLC",1,,
8,Apr 18 2023 1:07PM,261326,15,"Person & Covey, Inc.",1,,
9,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc.",34,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 2:03PM,261348,10,"Nextsource Biotechnology, LLC",1,,
1,Apr 18 2023 2:00PM,261346,19,Eywa Pharma Inc.,2,,
2,Apr 18 2023 1:53PM,261347,10,"Methapharm, Inc.",9,,
3,Apr 18 2023 1:51PM,261345,10,"Methapharm, Inc.",1,,
4,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 2:03PM,261348,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Apr 18 2023 2:00PM,261346,19,Eywa Pharma Inc.,2.0,NaN,NaN
2,Apr 18 2023 1:53PM,261347,10,"Methapharm, Inc.",9.0,NaN,NaN
3,Apr 18 2023 1:51PM,261345,10,"Methapharm, Inc.",1.0,NaN,NaN
4,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 2:03PM,261348,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Apr 18 2023 2:00PM,261346,19,Eywa Pharma Inc.,2.0,0.0,0.0
2,Apr 18 2023 1:53PM,261347,10,"Methapharm, Inc.",9.0,0.0,0.0
3,Apr 18 2023 1:51PM,261345,10,"Methapharm, Inc.",1.0,0.0,0.0
4,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3919851,39.0,10.0,1.0
15,1045222,37.0,1.0,0.0
19,1829112,35.0,24.0,5.0
20,522172,11.0,27.0,16.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3919851,39.0,10.0,1.0
1,15,1045222,37.0,1.0,0.0
2,19,1829112,35.0,24.0,5.0
3,20,522172,11.0,27.0,16.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,39.0,10.0,1.0
1,15,37.0,1.0,0.0
2,19,35.0,24.0,5.0
3,20,11.0,27.0,16.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,39.0
1,15,Released,37.0
2,19,Released,35.0
3,20,Released,11.0
4,10,Executing,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,1.0,0.0,5.0,16.0
Executing,10.0,1.0,24.0,27.0
Released,39.0,37.0,35.0,11.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,1.0,0.0,5.0,16.0
1,Executing,10.0,1.0,24.0,27.0
2,Released,39.0,37.0,35.0,11.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,1.0,0.0,5.0,16.0
1,Executing,10.0,1.0,24.0,27.0
2,Released,39.0,37.0,35.0,11.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()